Day 36 : 21.03.10

# Approaching (Almost) Any NLP Problem on Kaggle

https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

여기선 자연어 처리 문제에 대해 이야기하고자 한다. 우선 기본적인 첫 번째 모델을 만든 후 다른 변수를 사용해 개선한다. 또한 신경망이 얼마나 심층적으로 사용될 수 있는지 보고 일반적으로 앙상블에 대한 아이디어로 끝낼 것이다.

여기서 다룰 문제:
- tf-idf
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')

In [4]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [5]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [6]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


이 문제는 저자를 예측하는 문제다. 즉, text에 주어진 EAP, HPL, MWS를 예측해야한다. 간단히 말해서 text 분류는 3가지 클래스로 이루어진다.

이 특정 문제에 대해 Kaggle은 multi-class log-loss(다중 클래스 로그 손실)을 평가 지표로 지정했다.

In [7]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """ 로그 손실 메트릭의 다중 클래스 버전 """
    # actual : 실제 target 클래스를 포함하는 배열
    # predicted : 클래스 예측이 있는 행렬, 클래스당 하나의 확률
    
    # 이진 배열이 아닌 actual을 이진 배열로 변환
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i,val] = 1
        actual = actual2
        
    clip = np.clip(predicted, eps, 1-eps)  ### np.clip(array, min, max) : array의 min, max 범위 넘어가는 요소를 작으면 min, 크면 max로 변환
    rows = actual.shape[0]
    vsota = np.sum(actual*np.log(clip))
    
    return -1.0 / rows*vsota

scikit-learn의 LabelEncoder를 사용해 text 레이블을 정수로 변환 : 0,1,2

In [8]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

더 나아가기 전에 데이터를 training, validation 셋으로 나누는 것이 중요하다. scikit-learn의 model_selection 모듈에서 train_test_split을 사용한다.

In [10]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values,y,
                                                 stratify=y, random_state=42, test_size=0.1, shuffle=True)

In [11]:
print(xtrain.shape)
print(xvalid.shape)

(17621,)
(1958,)


## Building Basic Models

첫 번재 모델을 만들어보자.

첫 모델을 단순 TF-IDF(Term Frequency - Inverse Document Frequency) 후 단순 로지스틱 회귀분석이다.

In [19]:
# 항상 이 매개변수들로 해라. 거의 항상 이걸로 작동된다.
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                     token_pattern=r'\w{1,}', ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')
# training, test 셋에 TF-IDF 적합 (준지도학습)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [24]:
# TF-IDF에 단순 로지스틱 회귀분석 적합
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

AttributeError: 'str' object has no attribute 'decode'

0.626의 다중 클래스 로그 손실을 갖는 첫 모델을 생성했다. 하지만 더 좋은 점수를 위해 다른 데이터로 동일한 모델을 살펴보자.

TF-IDF를 사용하는 대신 단어 수를 변수로 사용할 수도 있다. 이 작업은 scikit-learn에서 CountVectorizer를 사용해 쉽게 할 수 있다.

In [26]:
ctv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')

# training, test 셋에 Count Vectorizer 적합 (준지도학습)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [27]:
# Counts에 단순 로지스틱 회귀 적합
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

AttributeError: 'str' object has no attribute 'decode'

첫 모델을 0.1점 향상시켰다!

다음으로, Naive Bayes 모델을 시도해보자. 다음 두 데이터셋에서 Naive bayes를 사용할 때 어떤 일이 일어나는지 살펴보자.

In [28]:
# TF-IDF에 단순 Naive Bayes 적합
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

logloss : 0.578


좋은 성능이지만 count에 대한 로지스틱회귀분석이 더 좋다. 대신 count 데이터에 이 모델을 사용하면 어떻게 될까?

In [29]:
# Counts에 단순 Naive Bayes 적합
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

logloss : 0.485


다음은 SVM을 적용해보자. SVM은 시간이 많이 걸리므로 적용하기 전 Singlular Value Decomposition을 사용해 TF-IDF의 변수 개수를 줄인다. 또한 SVM을 적용하기 전 데이터를 표준화해야한다.

In [30]:
# 120개 요소를 선택해 SVD 적용. 120-200 요소가 SVM 모델에 충분하다
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# SVD로 얻은 데이터를 스케일링. 스케일링 없이 재사용할 변수 이름 재서렂ㅇ
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [31]:
# 단순 SVM 적합
clf = SVC(C=1.0, probability=True)  # 확률이 필요하기 때문
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

logloss : 0.727


SVM이 이 데이터에서 성능이 좋지 않은 것 같다. 캐글에서 가장 유명한 알고리즘인 xgboost를 적용해보자.

In [32]:
# tf-idf에 xgboost 적합
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)  ### tocsc : 희소행렬로 만들기
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

c:\anaconda3\envs\study\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:29:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.781


In [35]:
import warnings
warnings.filterwarnings('ignore')
# tf-idf에 xgboost 적합
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
                       subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[23:32:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.772


In [36]:
# tf-idf svd 변수에 xgboost 적용
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[23:36:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.768


In [37]:
# tf-idf svd 변수에 xgboost 적합
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print('logloss : %0.3f' %multiclass_logloss(yvalid, predictions))

[23:42:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss : 0.786


XGBoost는 안 좋아보인다. 하지만 정확하지 않다. 아직 매개변수 최적화를 하지 않았다. 최적화 방법은 다음 섹션에서 설명한다.

### Grid Search

매개변수를 최적화하기 위한 기술이다. 그리 효과적이지 않지만 사용할 그리드를 알고 있으면 좋은 결과를 얻을 수 있다. 

이 섹션에서 로지스틱 회귀분석을 사용한 Grid Search에 대해 설명한다. Grid Search를 시작하기 전, 점수 계산 함수를 만들어야한다. 이는 scikit-learn의 make_scorer 함수를 사용한다.

In [38]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

다음은 파이프라인이 필요하다. 여기서 시연하기 위해, SVD, 스케일링, 로지스틱 회귀분석으로 구성된 파이프라인을 사용한다. 하나의 모듈보다 더 많은 모듈을 파이프라인에 배치하는 것이 이해하기 좋다.

In [39]:
# SVD 초기화
svd = TruncatedSVD()
# standard scaler 초기화
scl = preprocessing.StandardScaler()
# 로지스틱 회귀분석
lr_model = LogisticRegression()
# 파이프라인 생성
clf = pipeline.Pipeline([('svd',svd), ('scl',scl), ('lr',lr_model)])

다음으로 매개변수의 그리드가 필요하다.

In [40]:
param_grid = {'svd__n_components':[120,180],
             'lr__C':[0.1,1.0,10],
             'lr__penalty':['l1','l2']}

따라서 SVD의 경우 120개와 180개의 요소를 평가하고, 로지스틱회귀분석의 경우 lr과 l2 패널티로 C의 세 가지 다른 값을 평가한다. 이제 이러한 매개 변수에 대한 Grid Search를 할 수 있다.

In [41]:
# Grid Search 모델 초기화
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                    verbose=10, n_jobs=1, iid=True, refit=True, cv=2)

# Grid Search 모델 적합
model.fit(xtrain_tfv, ytrain)  # 여기선 전체 데이터를 사용할 수 있지만 xtrain만 사용한다
print('Best score : %0.3f' %model.best_score_)
print('Best parameters set :')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s: %r' %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=nan, total=   1.8s
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=nan, total=   2.4s
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................


...

[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=nan, total=   1.5s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   23.2s remaining:    0.0s


[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=nan, total=   1.1s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=nan, total=   2.0s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=nan, total=   2.0s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.775, total=   1.7s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.766, total=   1.6s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=180, score=-0.745, total=   2.2s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=180, score=-0.743, total=   

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   48.5s finished


Best score : -0.737
Best parameters set :
	lr__C: 0.1
	lr__penalty: 'l2'
	svd__n_components: 180


점수는 SVM 점수와 비슷하다. 이 기술은 아래와 같이 xgboost 또는 multinomial naive bayes를 finetuning하는 데 사용할 수 있다. 여기서 tf-idf 데이터를 사용한다.

In [44]:
nb_model = MultinomialNB()
clf = pipeline.Pipeline([('nb',nb_model)])  # pipleine 생성
param_grid = {'nb__alpha':[0.001,0.01,0.1,1,10,100]}  # parameter grid

# Grid Search 모델 초기화
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                    verbose=10, n_jobs=1, iid=True, refit=True, cv=2)
# Grid Search 모델 적합
model.fit(xtrain_tfv, ytrain)  # 전체 데이터 사용 가능하지만 xtrain만 사용
print('Best score : %0.3f' %model.best_score_)
print('Best parameters set :')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s : %r' %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.001 .................................................
[CV] .................... nb__alpha=0.001, score=-0.620, total=   0.0s
[CV] nb__alpha=0.001 .................................................
[CV] .................... nb__alpha=0.001, score=-0.641, total=   0.0s
[CV] nb__alpha=0.01 ..................................................
[CV] ..................... nb__alpha=0.01, score=-0.511, total=   0.0s
[CV] nb__alpha=0.01 ..................................................
[CV] ..................... nb__alpha=0.01, score=-0.523, total=   0.0s
[CV] nb__alpha=0.1 ...................................................
[CV] ...................... nb__alpha=0.1, score=-0.489, total=   0.0s
[CV] nb__alpha=0.1 ...................................................
[CV] ...................... nb__alpha=0.1, score=-0.495, total=   0.0s
[CV] nb__alpha=1 .....................................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s


[CV] ...................... nb__alpha=100, score=-1.067, total=   0.0s
[CV] nb__alpha=100 ...................................................
[CV] ...................... nb__alpha=100, score=-1.067, total=   0.0s
Best score : -0.492
Best parameters set :
	nb__alpha : 0.1


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished


원래 naive bayes 점수보다 8% 향상되었다. 

NLP 문제에서는 일반적으로 단어 벡터를 살펴본다. 단어 벡터를 통해 데이터에 대한 통찰력을 얻을 수 있다. 자세히 알아보자.

## Word Vectors